In [1]:
# Data processing
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

In [2]:
dfCCO = pd.read_csv(r".\Data\processed\CCO_processed.csv", index_col="invoice_month")
dfPYCO = pd.read_csv(r".\Data\processed\PYCO_processed.csv", index_col="invoice_month")
dfXCO = pd.read_csv(r".\Data\processed\XCO_processed.csv", index_col="invoice_month")

In [59]:
def build_model (trainX, trainY, epoch, bs):
    
    
    model = Sequential()
    model.add(LSTM(8, activation='tanh', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
    model.add(LSTM(4, activation='tanh', return_sequences=False))
#     model.add(Dropout(0.2))
    model.add(Dense(trainY.shape[1]))

    model.compile(optimizer='adam', loss='mse')
    
    es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=20)
    rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15)
    mcp = ModelCheckpoint(filepath='weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)
    
    model.fit(trainX, trainY, shuffle=True, epochs= epoch, verbose=0, validation_split=0.2, batch_size= bs)

    
    return model

In [60]:
def forecast_costs(df: pd.DataFrame) -> float:
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df.fillna(0)), columns=df.columns)
    df_shift = df_scaled.shift(1)
    y = df_scaled.total_cost.shift(-1).head(-1).tail(-1).values[..., None]
    df_array = df_scaled.tail(-1).head(-1).fillna(0).values
    df_shift_array = df_shift.tail(-1).head(-1).fillna(0).values
    # Forecaster is the last two months turned into a tensor so that we can create a final forecast in the end.
    forecaster = np.concatenate([df_scaled.tail(1).values[..., None], df_shift.tail(1).values[..., None]], axis=2)
    X = np.concatenate([df_array[..., None], df_shift_array[..., None]], axis=2)
    model = build_model(X, y, 200, 1)
    forecast = model.predict(forecaster)
    return scaler.inverse_transform(np.resize(forecast, df.shape))[0, 0]

In [39]:
PYCO_up = dfPYCO.iloc[8:13]
PYCO_up

,total_cost,Elastic Load Balancing cost,Elastic Load Balancing usage,Amazon Elastic Compute Cloud cost,Amazon Elastic Compute Cloud usage,Amazon Simple Storage Service cost,Amazon Simple Storage Service usage,AWS Key Management Service cost,AWS Key Management Service usage,Amazon Cognito cost,...,Amazon DocumentDB (with MongoDB compatibility) cost,Amazon DocumentDB (with MongoDB compatibility) usage,Amazon Virtual Private Cloud cost,Amazon Virtual Private Cloud usage,AWS Step Functions cost,AWS Step Functions usage,Amazon Registrar cost,Amazon Registrar usage,Amazon Glacier cost,Amazon Glacier usage
invoice_month,,,,,,,,,,,,,,,,,,,,,
2020-05-01,6489.349368,NaN,NaN,4584.858139,80250.791519,235.900572,2.695843e+07,0.043525,34510.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01,7482.256827,NaN,NaN,5136.041689,84290.524854,268.039829,2.724291e+07,0.062792,40932.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-01,7707.567863,NaN,NaN,5797.072250,87149.629996,287.486184,2.989641e+07,0.034850,31618.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-01,7731.745608,133.138050,5741.158456,5908.525521,82895.455643,297.854546,2.509456e+07,0.000000,19362.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-01,8486.255090,205.604257,8668.786392,6561.398514,76923.357193,301.621427,2.249013e+07,0.000000,6118.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
PYCO_down = dfPYCO.iloc[:5]
PYCO_down

,total_cost,Elastic Load Balancing cost,Elastic Load Balancing usage,Amazon Elastic Compute Cloud cost,Amazon Elastic Compute Cloud usage,Amazon Simple Storage Service cost,Amazon Simple Storage Service usage,AWS Key Management Service cost,AWS Key Management Service usage,Amazon Cognito cost,...,Amazon DocumentDB (with MongoDB compatibility) cost,Amazon DocumentDB (with MongoDB compatibility) usage,Amazon Virtual Private Cloud cost,Amazon Virtual Private Cloud usage,AWS Step Functions cost,AWS Step Functions usage,Amazon Registrar cost,Amazon Registrar usage,Amazon Glacier cost,Amazon Glacier usage
invoice_month,,,,,,,,,,,,,,,,,,,,,
2019-09-01,11868.213465,NaN,NaN,9759.034174,77207.321219,64.663870,1.237815e+07,0.000000,426.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-01,10742.465921,NaN,NaN,7612.544765,95662.106033,55.099083,1.790421e+07,0.000000,682.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-01,10572.923600,NaN,NaN,6631.922277,76201.274526,81.014411,1.540055e+07,0.000000,777.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-01,8721.969470,NaN,NaN,5609.078014,74923.884233,100.358831,2.055291e+07,0.000000,1063.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01,8752.106507,NaN,NaN,5360.513077,86255.990066,168.925449,3.888306e+07,0.092671,50891.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
PYCO_mixed = dfPYCO.iloc[5:10]
PYCO_mixed

,total_cost,Elastic Load Balancing cost,Elastic Load Balancing usage,Amazon Elastic Compute Cloud cost,Amazon Elastic Compute Cloud usage,Amazon Simple Storage Service cost,Amazon Simple Storage Service usage,AWS Key Management Service cost,AWS Key Management Service usage,Amazon Cognito cost,...,Amazon DocumentDB (with MongoDB compatibility) cost,Amazon DocumentDB (with MongoDB compatibility) usage,Amazon Virtual Private Cloud cost,Amazon Virtual Private Cloud usage,AWS Step Functions cost,AWS Step Functions usage,Amazon Registrar cost,Amazon Registrar usage,Amazon Glacier cost,Amazon Glacier usage
invoice_month,,,,,,,,,,,,,,,,,,,,,
2020-02-01,7094.260716,NaN,NaN,4695.577926,7.264822e+04,188.909749,4.359042e+07,0.074398,44800.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-01,8728.033331,NaN,NaN,5307.141640,1.296659e+06,226.842485,4.995520e+07,0.086564,48855.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-01,7154.925511,NaN,NaN,4636.989642,7.977151e+04,227.142632,3.254586e+07,0.030285,30096.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-01,6489.349368,NaN,NaN,4584.858139,8.025079e+04,235.900572,2.695843e+07,0.043525,34510.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01,7482.256827,NaN,NaN,5136.041689,8.429052e+04,268.039829,2.724291e+07,0.062792,40932.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
forecast_costs(PYCO_up)

7757.106

In [62]:
forecast_costs(PYCO_down)

10334.688

In [63]:
forecast_costs(PYCO_mixed)

6502.3403